In [1]:
import cv2,os,time,glob
os.sys.path.append('../utilities/')
from datetime import datetime
import numpy as np
import IPython.display as Disp
import support_utility_openvino
import matplotlib.pyplot as plt
import ipywidgets as widgets
from shapely.geometry import Polygon,Point


In [2]:
import imagesize
import xml.etree.ElementTree as ET


In [3]:
model_path = "/media/prateek/shared_space/Notebooks/git_repos/openvino_training_extensions/pytorch_toolkit/object_detection/custom_model/person_detection_1/outputs/openvino/alt_ssd_export/2020_03_18_person_detection.xml"

In [4]:
model_path = "/media/prateek/shared_space/DATA_N_MODELS/useful_models/helmet_vest/2021_05_05_helmet_vest/tensorflow/openvino_graph/INT8/frozen_inference_graph_taloja_veraval_CROP.xml"

In [5]:
detector_model = support_utility_openvino.async_infer()
output_support = support_utility_openvino.create_plot()

Available Devices :  ['CPU', 'GNA', 'GPU']


In [6]:
detector_model.load_model(model_path,"CPU")

OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image_tensor'])
model outputs :  dict_keys(['DetectionOutput'])


In [63]:
cap = cv2.VideoCapture(f)
while True:
    ret,frame = cap.read()
    thresh = variance_of_laplacian(frame)
    cv2.putText(frame,str(thresh),(30,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,cv2.LINE_AA)
    cv2.imshow("preview",frame)
    k = cv2.waitKey(1)
    if k==ord('q'):
        break
cv2.destroyAllWindows()
cap.release()

In [7]:
def variance_of_laplacian(image):
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()



def get_all_file_path(source_dir,ext='.csv'):
    """
    all images with csv extension exist in set of dirs
    """
    op =[]
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith(ext):
                
                 op.append(os.path.join(os.path.abspath(root), file))
    return op


In [9]:
files = get_all_file_path("/media/prateek/shared_space/DATA_N_MODELS/useful_vids/helmet_vest/hirakud/30-09-2021/cropped_frames/",".jpg")
project_name = "helmet_vest_hirakud_1"
thresh = .2

skip_frame = 0 ### number of frames for which you want to skip the video
image_num = 0
d = datetime.now()
image_folder_path = "images/"+project_name+"/"+str(d.year)+"_"+str(d.month).zfill(2)+"_"+str(d.day).zfill(2)+"_"+str(d.hour).zfill(2)+"/"
os.makedirs(image_folder_path,exist_ok= True)
label_dict = {1:"helmet",2:"no_helmet",3:"vest",4:"no_vest",5:"worker"}

annotation = ET.Element("annotations")
detector_model.reinit_model()

for f in files:
    # images
    frame = cv2.imread(f)
    #videos
    
    # cap = cv2.VideoCapture(f)
    # while True:
    #     try:
    #         cap.set(1,int(cap.get(1))+skip_frame)
    #     except Exception as e:
    #         print(e)
    #     ret,frame = cap.read()
    #     if not ret:
    #         break
    detector_model.predict(frame)
    if detector_model.frame_processed>=detector_model.num_requests:
        frame, attrs, res = detector_model.postprocess_op()
        res_filt,bboxes = output_support.res2bbox(res[0],(frame.shape[1],frame.shape[0]),thresh=thresh)
        if len(bboxes)>0:
            frame_op = output_support.label_obj_detection(frame, res[0],thresh,list(label_dict.values()))
            cv2.imshow("preview",frame_op)
            k = cv2.waitKey(1)
            if k==ord('q'):
                cv2.destroyAllWindows()
                break
            filename = os.path.join(image_folder_path,str(image_num).zfill(6)+".jpg")
            cv2.imwrite(filename,frame)
            class_ids = res_filt[:,1].astype(int)
            height,width = frame.shape[:2]
            image = ET.Element("image")
            image.set("name",str(image_num).zfill(6)+".jpg")
            image.set("id" ,str(image_num))
            image.set("width",str(width))
            image.set("height",str(height))
            for idx,b in enumerate(bboxes):
                box = ET.SubElement(image,"box")
                box.set("label",label_dict[class_ids[idx]])
                box.set("occluded","0")
                box.set("xtl",str(b[0]))
                box.set("ytl",str(b[1]))
                box.set("xbr",str(b[2]))
                box.set("ybr",str(b[3]))
            annotation.append(image)
            image_num+=1
    
                

                
                
               
        

mydata = ET.tostring(annotation)
myfile = open(os.path.join("images/"+project_name+"/","annotation.xml"), "wb")
myfile.write(mydata)
myfile.close()
cv2.destroyAllWindows()

In [12]:
files

[]

### annotation for frames only not for video

In [ ]:
#!/usr/bin/env python
# coding:utf-8
import imagesize
import xml.etree.ElementTree as ET
annotation = ET.Element("annotations")
for row in df.iterrows():
    width, height = imagesize.get(row[1]["filename"])
    image = ET.Element("image")
    image.set("name",row[1]["filename"].replace("/media/prateek/prateek_space/dataset/UTCL","/home/computer_vision/Downloads/Notebooks/UTCL"))
    image.set("id" ,str(row[0]))
    image.set("width",str(width))
    image.set("height",str(height))
    if not np.isnan(row[1]["helmet_coords"]).any():
        coords = row[1]["helmet_coords"]
        box = ET.SubElement(image,"box")
        box.set("label",row[1]["helmet_label"])
        box.set("occluded","0")
        box.set("xtl",str(coords[0]))
        box.set("ytl",str(coords[1]))
        box.set("xbr",str(coords[2]))
        box.set("ybr",str(coords[3]))
    if not np.isnan(row[1]["vest_coords"]).any():
        coords = row[1]["vest_coords"]
        box = ET.SubElement(image,"box")
        box.set("label",row[1]["vest_label"])
        box.set("occluded","0")
        box.set("xtl",str(coords[0]))
        box.set("ytl",str(coords[1]))
        box.set("xbr",str(coords[2]))
        box.set("ybr",str(coords[3]))
    annotation.append(image)
    